In [31]:
import pandas as pd

df = pd.read_csv("../data/processed/clean_data.csv")
#df = df.drop("Unnamed: 0", axis = 1)
df

,Impressions,From Home,From Hashtags,From Explore,From Other,Saves,Comments,Shares,Likes,Profile Visits,...,HashtagCount,HashtagDensity,FromHomeRatio,FromHashtagsRatio,FromExploreRatio,FromOtherRatio,IsWeekend,Month,HourOfDay,NumOfHashtags
0,3920,2586,1028,619,56,98,9,5,162,35,...,22,0.111111,0.659694,0.262245,0.157908,0.014286,0,5,22,22
1,5394,2727,1838,1174,78,194,7,14,224,48,...,18,0.055556,0.505562,0.340749,0.217649,0.014461,0,5,3,18
2,4021,2085,1188,0,533,41,11,1,131,62,...,18,0.071429,0.518528,0.295449,0.000000,0.132554,1,1,6,18
3,4528,2700,621,932,73,172,10,7,213,23,...,11,0.058824,0.596290,0.137147,0.205830,0.016122,0,6,16,11
4,2518,1704,255,279,37,96,5,4,123,8,...,29,0.062500,0.676728,0.101271,0.110802,0.014694,0,3,17,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,13700,5185,3041,5352,77,573,2,38,373,73,...,17,0.166667,0.378467,0.221971,0.390657,0.005620,0,8,18,17
115,5731,1923,1368,2266,65,135,4,1,148,20,...,18,0.037037,0.335544,0.238702,0.395393,0.011342,0,11,3,18
116,4139,1133,1538,1367,33,36,0,1,92,34,...,18,0.050000,0.273738,0.371587,0.330273,0.007973,0,2,22,18
117,32695,11815,3147,17414,170,1095,2,75,549,148,...,17,0.166667,0.361370,0.096253,0.532620,0.005200,0,10,4,17


In [32]:
df.columns

Index(['Impressions', 'From Home', 'From Hashtags', 'From Explore',
       'From Other', 'Saves', 'Comments', 'Shares', 'Likes', 'Profile Visits',
       'Follows', 'Caption', 'Hashtags', 'Timestamp', 'Content Type',
       'CaptionLength', 'DayOfWeek', 'EngagementRate', 'HashtagCount',
       'HashtagDensity', 'FromHomeRatio', 'FromHashtagsRatio',
       'FromExploreRatio', 'FromOtherRatio', 'IsWeekend', 'Month', 'HourOfDay',
       'NumOfHashtags'],
      dtype='object')

In [33]:
import sys, os

sys.path.append(os.path.abspath("../"))
# from utils.feature_engineering import add_hashtags_count
# add_hashtags_count(df)
# df

In [34]:
df.columns

Index(['Impressions', 'From Home', 'From Hashtags', 'From Explore',
       'From Other', 'Saves', 'Comments', 'Shares', 'Likes', 'Profile Visits',
       'Follows', 'Caption', 'Hashtags', 'Timestamp', 'Content Type',
       'CaptionLength', 'DayOfWeek', 'EngagementRate', 'HashtagCount',
       'HashtagDensity', 'FromHomeRatio', 'FromHashtagsRatio',
       'FromExploreRatio', 'FromOtherRatio', 'IsWeekend', 'Month', 'HourOfDay',
       'NumOfHashtags'],
      dtype='object')

In [35]:
#df.to_csv('../data/processed/clean_data.csv', index=False)

In [36]:
df["Content Type"].isnull().sum()

0

In [37]:
df["Caption"].isnull().sum()

0

In [38]:
df["Hashtags"].isnull().sum()

0

In [39]:
X = df.drop(columns=['Impressions', 'Timestamp', 'NumOfHashtags', 'Profile Visits', 'Saves', 'Shares',
                     'From Home', 'From Hashtags', 'From Explore', 'From Other', 'Comments', 'Follows', 'Likes',
                     'FromHomeRatio', 'FromHashtagsRatio', 'EngagementRate', 'FromExploreRatio', 
                     'FromOtherRatio', 'Content Type', 'Caption', 'Hashtags'])


In [41]:
import numpy as np 
import pandas as pd
y = np.log1p(df["Impressions"])  # log(1 + x)

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,
                                                      random_state=42)

In [50]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
scale = Pipeline([
 ("scaler", StandardScaler())

])
scale_features = ['CaptionLength', 'DayOfWeek', 'Month',
                    'HourOfDay', 'HashtagCount']


passthrough_features = ['HashtagDensity', 'IsWeekend']

preprocessor = ColumnTransformer(
    transformers=[
        ('scale', StandardScaler(), scale_features)
])

In [54]:
#sys.path.append(os.path.abspath("../"))
from sklearn.linear_model import Ridge, PassiveAggressiveRegressor
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error

#from src.pipeline import pipeline
#rmse_scores = np.sqrt(-scores)

par = PassiveAggressiveRegressor(random_state=42)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', par)
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

rmse = root_mean_squared_error(y_true=y_test, y_pred=y_pred)
r_2 = r2_score(y_true=y_test, y_pred=y_pred)


print("rmse", rmse)
print("r2", r_2)


rmse 0.7376532062575386
r2 -0.131330582440772
